In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw8.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 8: Time series
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

## Submission instructions
<hr>
rubric={points:4}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Upload the .ipynb file to Gradescope.
- **If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb.** 
- Make sure that your plots/output are rendered properly in Gradescope.

<br><br>

## Exercise 1: time series prediction

In this exercise we'll be looking at a [dataset of avocado prices](https://www.kaggle.com/neuromusic/avocado-prices). You should start by downloading the dataset. We will be forcasting average avocado price for the next week. 

In [3]:
df = pd.read_csv("data/avocado.csv", parse_dates=["Date"], index_col=0)
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
df.shape

(18249, 13)

In [5]:
df["Date"].min()

Timestamp('2015-01-04 00:00:00')

In [6]:
df["Date"].max()

Timestamp('2018-03-25 00:00:00')

It looks like the data ranges from the start of 2015 to March 2018 (~2 years ago), for a total of 3.25 years or so. Let's split the data so that we have a 6 months of test data.

In [7]:
split_date = '20170925'
df_train = df[df["Date"] <= split_date]
df_test  = df[df["Date"] >  split_date]

In [8]:
assert len(df_train) + len(df_test) == len(df)

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 How many time series? 
rubric={points:4}

In the Rain is Australia dataset from lecture, we had different measurements for each Location. What about this dataset: for which categorical feature(s), if any, do we have separate measurements? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [9]:
# Sorting the dataframe by date 

date_sorted = df_train.sort_values(["Date", "region"])
# region_sorted = df_train.sort_values("region")
date_sorted

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
51,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.00,conventional,2015,Albany
51,2015-01-04,1.79,1373.95,57.42,153.88,0.00,1162.65,1162.65,0.00,0.00,organic,2015,Albany
51,2015-01-04,1.00,435021.49,364302.39,23821.16,82.15,46815.79,16707.15,30108.64,0.00,conventional,2015,Atlanta
51,2015-01-04,1.76,3846.69,1500.15,938.35,0.00,1408.19,1071.35,336.84,0.00,organic,2015,Atlanta
51,2015-01-04,1.08,788025.06,53987.31,552906.04,39995.03,141136.68,137146.07,3990.61,0.00,conventional,2015,BaltimoreWashington
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,2017-09-24,1.94,1125443.38,116338.61,237876.77,1182.85,769536.90,639021.62,130401.43,113.85,organic,2017,TotalUS
14,2017-09-24,1.59,3842682.36,1450858.85,1006684.68,42995.08,1342143.75,764285.55,575719.82,2138.38,conventional,2017,West
14,2017-09-24,2.40,111321.19,17650.08,22295.44,42.34,71333.33,40018.26,31209.23,105.84,organic,2017,West
14,2017-09-24,1.22,584045.70,363596.75,99990.28,415.70,120042.97,68799.49,51236.81,6.67,conventional,2017,WestTexNewMexico


After sorting by date we saw on the same date there are several measurements (several rows) hence there are multiple timeseries in our dataset. To further investigate, we sorted by the regions and saw on the same date and for the same date, there are two measurements based on the types. Therefore, we have a timeseries for each region and type of avocado. 

In [10]:
print(df_train["region"].unique())
print(df_train["type"].unique())

print(len(df_train["region"].unique()))
print(len(df_train["type"].unique()))

['Albany' 'Atlanta' 'BaltimoreWashington' 'Boise' 'Boston'
 'BuffaloRochester' 'California' 'Charlotte' 'Chicago' 'CincinnatiDayton'
 'Columbus' 'DallasFtWorth' 'Denver' 'Detroit' 'GrandRapids' 'GreatLakes'
 'HarrisburgScranton' 'HartfordSpringfield' 'Houston' 'Indianapolis'
 'Jacksonville' 'LasVegas' 'LosAngeles' 'Louisville' 'MiamiFtLauderdale'
 'Midsouth' 'Nashville' 'NewOrleansMobile' 'NewYork' 'Northeast'
 'NorthernNewEngland' 'Orlando' 'Philadelphia' 'PhoenixTucson'
 'Pittsburgh' 'Plains' 'Portland' 'RaleighGreensboro' 'RichmondNorfolk'
 'Roanoke' 'Sacramento' 'SanDiego' 'SanFrancisco' 'Seattle'
 'SouthCarolina' 'SouthCentral' 'Southeast' 'Spokane' 'StLouis' 'Syracuse'
 'Tampa' 'TotalUS' 'West' 'WestTexNewMexico']
['conventional' 'organic']
54
2


In [11]:
# Exploring how many regions we have in the dataset and further exploring the different time series

albany = df_train[df_train["region"] == "Albany"]
albany_conven = albany[albany["type"] == "conventional"]
albany_organic = albany[albany["type"] == "organic"]

print(albany_conven)
print(albany_organic)

         Date  AveragePrice  Total Volume     4046       4225     4770  \
0  2015-12-27          1.33      64236.62  1036.74   54454.85    48.16   
1  2015-12-20          1.35      54876.98   674.28   44638.81    58.33   
2  2015-12-13          0.93     118220.22   794.70  109149.67   130.50   
3  2015-12-06          1.08      78992.15  1132.00   71976.41    72.58   
4  2015-11-29          1.28      51039.60   941.48   43838.39    75.78   
..        ...           ...           ...      ...        ...      ...   
48 2017-01-29          1.31      95424.59  3844.62   78315.15   484.56   
49 2017-01-22          1.59     128679.24  4119.94  111173.08  2191.71   
50 2017-01-15          1.55      88526.26  3327.65   71956.77   607.03   
51 2017-01-08          1.55      91728.18  3355.47   75641.23    56.91   
52 2017-01-01          1.47     129948.23  4845.77  117027.41   200.36   

    Total Bags  Small Bags  Large Bags  XLarge Bags          type  year  \
0      8696.87     8603.62       93.

In [12]:
df_train[df_train["Date"] == "2015-12-27"]

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.00,conventional,2015,Albany
0,2015-12-27,0.99,386100.49,292097.36,27350.92,297.90,66354.31,48605.95,17748.36,0.00,conventional,2015,Atlanta
0,2015-12-27,1.17,596819.40,40450.49,394104.02,17353.79,144911.10,142543.88,2367.22,0.00,conventional,2015,BaltimoreWashington
0,2015-12-27,0.97,62909.69,30482.25,2971.94,5894.40,23561.10,23520.19,5.69,35.22,conventional,2015,Boise
0,2015-12-27,1.13,450816.39,3886.27,346964.70,13952.56,86012.86,85913.60,99.26,0.00,conventional,2015,Boston
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2015-12-27,1.54,1652.19,0.00,73.22,0.00,1578.97,1336.27,242.70,0.00,organic,2015,Syracuse
0,2015-12-27,1.63,2161.84,874.75,17.54,0.00,1269.55,1216.67,52.88,0.00,organic,2015,Tampa
0,2015-12-27,1.52,549787.59,89709.92,206198.62,5836.04,248043.01,142262.93,105780.08,0.00,organic,2015,TotalUS
0,2015-12-27,1.46,142710.36,29880.32,48416.71,38.63,64374.70,17464.54,46910.16,0.00,organic,2015,West


For our analysis, we picked the first date from the dataframe. For this particular date we can there are 108 different points. Inspecting these 108 points we can see that each pair of type and region has a different time series associated with it. This is supported by the fact that we have 54 unique regions and 2 unique types, giving us 108 pairs of region and type combinations. Overall, we have 108 timeseries in our data, for the two categorical variables - region and type.  

<!-- BEGIN QUESTION -->

### 1.2 Equally spaced measurements? 
rubric={points:4}

In the Rain in Australia dataset, the measurements were generally equally spaced but with some exceptions. How about with this dataset? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.2
    
</div>

To analyze spacing, we iteratively went through each region and type combination. 
From the results we can see that only WestTexNewMexico and organic type has an irregularly spaced data. 

They also had 140 rows compared to all region and type that has 143 rows.  
Specifically, as shown below, the dates betweeen 2015-11-29 and 2015-12-13 is irregularly spaced compared to all other time points, and also 2017-06-11 and 2017-07-02 are also irregular compared to all other time points. 

In [13]:
df_train["Date"]

0    2015-12-27
1    2015-12-20
2    2015-12-13
3    2015-12-06
4    2015-11-29
        ...    
46   2017-01-29
47   2017-01-22
48   2017-01-15
49   2017-01-08
50   2017-01-01
Name: Date, Length: 15441, dtype: datetime64[ns]

In [14]:
# temp = df_train[np.logical_and(df_train["region"] == "Albany", df_train["type"] == "conventional")]
# dates = np.sort(temp["Date"])
# prev = dates[0]
# diff = []
# for i in range(len(dates)):
#     diff.append(dates[i] - prev)
#     prev = dates[i]
# # diff = diff[1:]
# # print(len(set(diff)) == 1)

# print(len(diff))
result = []

for region in df_train["region"].unique():
    for tp in df_train["type"].unique():
        temp = df_train[np.logical_and(df_train["region"] == region, df_train["type"] == tp)]
        dates = np.sort(temp["Date"])
        prev = dates[0]
        diff = []
        for i in range(1, len(dates)):
            day = (dates[i] - prev)
            diff.append(day.astype(int))
            prev = dates[i]
            
            if day != diff[0]:
                print(dates[i])
                print(i)
                
        result.append(len(set(diff)) == 1)
        if (len(set(diff)) == 1) == False:
            print(region, tp)
            print(len(temp))
print(all(result))

2015-12-13T00:00:00.000000000
48
2017-07-02T00:00:00.000000000
127
WestTexNewMexico organic
140
False


In [15]:
temp_df = df_train[np.logical_and(df_train["region"] == "WestTexNewMexico", df_train["type"] == "organic")]
print(temp_df[47:49])
print(temp_df[126:129])

         Date  AveragePrice  Total Volume     4046     4225  4770  Total Bags  \
47 2015-01-25          1.63       7324.06  1934.46  3032.72   0.0     2356.88   
48 2015-01-18          1.71       5508.20  1793.64  2078.72   0.0     1635.84   

    Small Bags  Large Bags  XLarge Bags     type  year            region  
47      2320.0       36.88          0.0  organic  2015  WestTexNewMexico  
48      1620.0       15.84          0.0  organic  2015  WestTexNewMexico  
         Date  AveragePrice  Total Volume     4046     4225   4770  \
37 2017-04-02          1.04      28803.50  2551.72  2586.16  11.61   
38 2017-03-26          1.11      32073.43  3041.33  3451.21  66.76   
39 2017-03-19          1.11      26767.63  4255.75  3855.69  11.67   

    Total Bags  Small Bags  Large Bags  XLarge Bags     type  year  \
37    23654.01    23267.01      387.00          0.0  organic  2017   
38    25514.13    25191.62      322.51          0.0  organic  2017   
39    18644.52    13278.06     5366.46  

In [16]:
# Checking number of rows for all other region and type 

len_ts = []
for region in df_train["region"].unique():
    for tp in df_train["type"].unique():
        length = len(df_train[np.logical_and(df_train["region"] == region, df_train["type"] == tp)])
        len_ts.append(length)
        
        if length != len_ts[0]:
            print(region)
            print(tp)

print(len_ts)


        
        

WestTexNewMexico
organic
[143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 143, 140]


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Interpreting regions 
rubric={points:4}

In the Rain is Australia dataset, each location was a different place in Australia. For this dataset, look at the names of the regions. Do you think the regions are also all distinct, or are there overlapping regions? Justify your answer by referencing the data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

In [17]:
print((df_train["region"].unique()))

['Albany' 'Atlanta' 'BaltimoreWashington' 'Boise' 'Boston'
 'BuffaloRochester' 'California' 'Charlotte' 'Chicago' 'CincinnatiDayton'
 'Columbus' 'DallasFtWorth' 'Denver' 'Detroit' 'GrandRapids' 'GreatLakes'
 'HarrisburgScranton' 'HartfordSpringfield' 'Houston' 'Indianapolis'
 'Jacksonville' 'LasVegas' 'LosAngeles' 'Louisville' 'MiamiFtLauderdale'
 'Midsouth' 'Nashville' 'NewOrleansMobile' 'NewYork' 'Northeast'
 'NorthernNewEngland' 'Orlando' 'Philadelphia' 'PhoenixTucson'
 'Pittsburgh' 'Plains' 'Portland' 'RaleighGreensboro' 'RichmondNorfolk'
 'Roanoke' 'Sacramento' 'SanDiego' 'SanFrancisco' 'Seattle'
 'SouthCarolina' 'SouthCentral' 'Southeast' 'Spokane' 'StLouis' 'Syracuse'
 'Tampa' 'TotalUS' 'West' 'WestTexNewMexico']


While most of the entries in "region" are cities, there are a few exceptions such as "California", "SouthCarolina", and "Northeast" that are states and larger geographical area. These exceptions overlap with other cities, for example "California" overlaps with 'SanDiego', 'SanFrancisco', and other cities in the state of California. 

<!-- END QUESTION -->

<br><br>

We will use the entire dataset despite any location-based weirdness uncovered in the previous part.

We will be trying to forecast the avocado price. The function below is adapted from Lecture 20, with some improvements.

In [18]:
def create_lag_feature(df, orig_feature, lag, groupby, new_feature_name=None, clip=False):
    """
    Creates a new feature that's a lagged version of an existing one.
    
    NOTE: assumes df is already sorted by the time columns and has unique indices.
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The dataset.
    orig_feature : str
        The column name of the feature we're copying
    lag : int
        The lag; negative lag means values from the past, positive lag means values from the future
    groupby : list
        Column(s) to group by in case df contains multiple time series
    new_feature_name : str
        Override the default name of the newly created column
    clip : bool
        If True, remove rows with a NaN values for the new feature
    
    Returns
    -------
    pandas.core.frame.DataFrame
        A new dataframe with the additional column added.
        
    TODO: could/should simplify this function by using `df.shift()`
    """
        
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = "%s_lag%d" % (orig_feature, -lag)
        else:
            new_feature_name = "%s_ahead%d" % (orig_feature, lag)
    
    new_df = df.assign(**{new_feature_name : np.nan})
    for name, group in new_df.groupby(groupby):        
        if lag < 0: # take values from the past
            new_df.loc[group.index[-lag:],new_feature_name] = group.iloc[:lag][orig_feature].values
        else:       # take values from the future
            new_df.loc[group.index[:-lag], new_feature_name] = group.iloc[lag:][orig_feature].values
            
    if clip:
        new_df = new_df.dropna(subset=[new_feature_name])
        
    return new_df

We first sort our dataframe properly:

In [19]:
df_sort = df.sort_values(by=["region", "type", "Date"]).reset_index(drop=True)
df_sort

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico
18247,2018-03-18,1.56,15896.38,2055.35,1499.55,0.00,12341.48,12114.81,226.67,0.0,organic,2018,WestTexNewMexico


We then call `create_lag_feature`. This creates a new column in the dataset `AveragePriceNextWeek`, which is the following week's `AveragePrice`. We have set `clip=True` which means it will remove rows where the target would be missing.

In [20]:
df_hastarget = create_lag_feature(df_sort, "AveragePrice", +1, ["region", "type"], "AveragePriceNextWeek", clip=True)
df_hastarget

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18243,2018-02-18,1.56,17597.12,1892.05,1928.36,0.00,13776.71,13553.53,223.18,0.0,organic,2018,WestTexNewMexico,1.57
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico,1.54
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico,1.56
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico,1.56


Our goal is to predict `AveragePriceNextWeek`. 

Let's split the data:

In [21]:
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 `AveragePrice` baseline 
rubric={points:4}

Soon we will want to build some models to forecast the average avocado price a week in advance. Before we start with any ML though, let's try a baseline. Previously we used `DummyClassifier` or `DummyRegressor` as a baseline. This time, we'll do something else as a baseline: we'll assume the price stays the same from this week to next week. So, we'll set our prediction of "AveragePriceNextWeek" exactly equal to "AveragePrice", assuming no change. That is kind of like saying, "If it's raining today then I'm guessing it will be raining tomorrow". This simplistic approach will not get a great score but it's a good starting point for reference. If our model does worse that this, it must not be very good. 

Using this baseline approach, what $R^2$ do you get on the train and test data?

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Type your answer here, replacing this text._

In [22]:
# Calculating R2 on training data 
r2_score(df_train["AveragePriceNextWeek"], df_train["AveragePrice"])

0.8285800937261841

In [23]:
# Calculating R2 on testing data 
r2_score(df_test["AveragePriceNextWeek"], df_test["AveragePrice"])

0.7631780188583048

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Forecasting average avocado price
rubric={points:10}

Now that the baseline is done, let's build some models to forecast the average avocado price a week later. Experiment with a few approachs for encoding the date. Justify the decisions you make. Which approach worked best? Report your test score and briefly discuss your results.

Benchmark: you should be able to achieve $R^2$ of at least 0.79 on the test set. I got to 0.80, but not beyond that. Let me know if you do better!

Note: because we only have 2 splits here, we need to be a bit wary of overfitting on the test set. Try not to test on it a ridiculous number of times. If you are interested in some proper ways of dealing with this, see for example sklearn's [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html), which is like cross-validation for time series data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Type your answer here, replacing this text._

In [24]:
def get_season(month):
    SUMMER_MONTHS = ["June", "July", "August"] 
    SPRING_MONTHS = ["March", "April", "May"]
    WINTER_MONTHS = ["December", "January", "February"]
    AUTUMN_MONTHS = ["September", "October", "November"]
    if month in WINTER_MONTHS:
        return "Winter"
    elif month in AUTUMN_MONTHS:
        return "Autumn"
    elif month in SUMMER_MONTHS:
        return "Summer"
    else:
        return "Fall"

In [25]:
df_train = df_train.assign(Month=df_train["Date"].apply(lambda x: x.month_name()))
df_test = df_test.assign(Month=df_test["Date"].apply(lambda x: x.month_name()))

df_train = df_train.assign(Season=df_train["Date"].apply(lambda x: get_season(x.month_name())))
df_test = df_test.assign(Season=df_test["Date"].apply(lambda x: get_season(x.month_name())))

X_train = df_train.drop(columns= ["AveragePriceNextWeek"])
y_train = df_train["AveragePriceNextWeek"]
X_test = df_test.drop(columns= ["AveragePriceNextWeek"])
y_test = df_test["AveragePriceNextWeek"]

In [26]:
numeric_features = ["AveragePrice", "Total Volume", "4046", "4225", "4770", "Small Bags", "Large Bags", "XLarge Bags"]
categorical_features = ["type", "year", "region", "Season"]
drop_features = ["Date","Month"]

In [27]:
numeric_transformer = make_pipeline(
        SimpleImputer(strategy="median"), StandardScaler()
    )
categorical_transformer = make_pipeline(
        SimpleImputer(strategy="constant", fill_value="missing"),
        OneHotEncoder(handle_unknown="ignore", sparse=False),
    )
preprocessor = make_column_transformer(
        (numeric_transformer, numeric_features),
        (categorical_transformer, categorical_features),
        ("drop", drop_features),
    )

In [28]:
#OHE on season
pipe = make_pipeline(preprocessor, Ridge())
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.7937165719422217

In [29]:
categorical_features = ["type", "year", "region", "Month"]
drop_features = ["Date","Season"]
numeric_transformer = make_pipeline(
        SimpleImputer(strategy="median"), StandardScaler()
    )
categorical_transformer = make_pipeline(
        SimpleImputer(strategy="constant", fill_value="missing"),
        OneHotEncoder(handle_unknown="ignore", sparse=False),
    )
preprocessor = make_column_transformer(
        (numeric_transformer, numeric_features),
        (categorical_transformer, categorical_features),
        ("drop", drop_features),
    )

In [30]:
#OHE on month
pipe = make_pipeline(preprocessor, Ridge())
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.8018263014781949

We chose to approach the encoding of "Date" in two different ways: one hot encoding based on season and month. We hypothesized that the price of avocado might change based on season. Therefore, we performed feature engineering to extract seasons from the "Date" column. After OHE on season, which gave us the R^2 score of 0.79. We then tried OHE on month because we thought the price of avocado could also depend on individual month as opposed to grouping into seasons, and it did give us a little improvement with a R^2 of 0.80. 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: very short answer questions

Each question is worth 2 points.

<!-- BEGIN QUESTION -->

### 2.1 Time series

rubric={points:4}

The following questions pertain to Lecture 20 on time series data:

1. Sometimes a time series has missing time points or, worse, time points that are unequally spaced in general. Give an example of a real world situation where the time series data would have unequally spaced time points.
2. In class we discussed two approaches to using temporal information: encoding the date as one or more features, and creating lagged versions of features. Which of these (one/other/both/neither) two approaches would struggle with unequally spaced time points? Briefly justify your answer.

<div class="alert alert-warning">

Solution_2.1
    
</div>

### Answer 2.1.1

Unequally spaced time points might occur due to various reasons: 
(1) a server might not be up all the time eg in case they need to be turned off for backing up so if the database with that server was recording transactions, it might have time points missing
(2) during holidays markets might be closed so it is difficult to collect measurements for customers during those time points (for eg if the experiment is observing customer's behaviour)
(3) if natural disasters are measured, they will be collected at irregularly spaced time points since they dont occur regularly (inspired by the https://en.wikipedia.org/wiki/Unevenly_spaced_time_series)


### Answer 2.1.2

Lag versions of feature approach have the most chance to suffer from unequally spaced time points. If data is missing or is unequally spaced in one of the original features, shifting the data points will run into issues of inaccuracy. For example in our avocado price, if a week is missing, we can no longer accurately calculate the "Average Price Next Week" feature for the previous week. Thus, we lose information or accuracy by trying to "fix" the unequally spaced time points.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Survival analysis
rubric={points:6}

The following questions pertain to Lecture 21 on survival analysis. We'll consider the use case of customer churn analysis.

1. What is the problem with simply labeling customers are "churned" or "not churned" and using standard supervised learning techniques?
2. Consider customer A who just joined last week vs. customer B who has been with the service for a year. Who do you expect will leave the service first: probably customer A, probably customer B, or we don't have enough information to answer?
3. If a customer's survival function is almost flat during a certain period, how do we interpret that?

<div class="alert alert-warning">

Solution_2.2
    
</div>

### Answer 2.2.1

We don't know whether the customer will churn after the collection period ends, so we cannot label it churned/not churned. Our output in these problems are right censored ie we donot know the customers churning or not churning after the study ends which restricts us from using standard supervised machine learning techniques. If all customers are labelled as churned at the end, our analysis will overestimate churning within that time and if all customers are labelled as not churned at the end, our analysis will overestimate not churning. 
Hence our main issue is that we donot know information on customers once the study collection period ends.

### Answer 2.2.2

According to the general survival probability vs time plot we saw in lecture, there is a high chance that a customer who has been with the service for long has a higher chance of churning since generally as time increases survival chances decreases. However, we need more information to analyze individual Customer behaviour since we need more demographic information from the individual person. For example - someone who just joined might be on a free trial (Customer A) vs a loyal customer (like customer B) so the person who joined recently might leave soon after the free trial ends. Conversely, someone who has been with the service for a long time might have their membership with the service ending soon vs who joined just now might have bought a year long membership so then Customer B will have higher chance of leaving. 

Thus we donot have enough information for a confirmed answer

### Answer 2.2.3

It means their survival probability does not change during that time period. Thus it can be interpreted that for the almost flat region of time, the customer has little chance of leaving.
For example, the time period here could be duration of their membership with the service and so the flat curve can show that once a customer buys a membership with the service, they will stay with the service for the entire duration of their membership. 

Note: Code used in Q1 was referenced from lecture on time series

<!-- END QUESTION -->

<br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
4. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

![](img/eva-well-done.png)